In [1]:
# !pip install ../input/kerasapplications/ > /dev/null
# !pip install ../input/efficientnet-keras-source-code/ > /dev/null

In [2]:
# cv = {'igm':[{'id':'hello', 'target':'1', 'family':1},{'id':'hello', 'target':'1', 'family':2}, {'id':'hello', 'target':'1', 'family':4}, {'id':'hello', 'target':'1', 'family':3}]}

In [3]:
# for igm , pr in cv.items():
#     cv[igm] = [sorted(pr[i].items(), key = lambda x: x['family']) for i in range(4)]

In [4]:
# import gc
# import os
# import math
# import random
# import re
# import warnings
# from pathlib import Path
# from PIL import Image
# from typing import Optional, Tuple

# import efficientnet.tfkeras as efn
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from scipy import spatial
# from sklearn.preprocessing import normalize
# from tqdm import tqdm

# DATADIR = Path("../input/landmark-retrieval-2021/")
# TEST_IMAGE_DIR = DATADIR / "test"
# TRAIN_IMAGE_DIR = DATADIR / "index"

# N_CLASSES = 81313

# import time

# from contextlib import contextmanager


# @contextmanager
# def timer(name):
#     t0 = time.time()
#     print(f"[{name}]")
#     yield
#     print(f'[{name}] done in {time.time() - t0:.0f} s')
# def set_seed(seed=42):
#     random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)
#     np.random.seed(seed)
#     tf.random.set_seed(seed)


# set_seed(1213)
# def auto_select_accelerator():
#     try:
#         tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#         tf.config.experimental_connect_to_cluster(tpu)
#         tf.tpu.experimental.initialize_tpu_system(tpu)
#         strategy = tf.distribute.experimental.TPUStrategy(tpu)
#         print("Running on TPU:", tpu.master())
#     except ValueError:
#         strategy = tf.distribute.get_strategy()
#     print(f"Running on {strategy.num_replicas_in_sync} replicas")
#     return strategy
# strategy = auto_select_accelerator()
# REPLICAS = strategy.num_replicas_in_sync
# AUTO = tf.data.experimental.AUTOTUNE

# class GeM(tf.keras.layers.Layer):
#     def __init__(self, pool_size, init_norm=3.0, normalize=False, **kwargs):
#         self.pool_size = pool_size
#         self.init_norm = init_norm
#         self.normalize = normalize

#         super(GeM, self).__init__(**kwargs)

#     def get_config(self):
#         config = super().get_config().copy()
#         config.update({
#             'pool_size': self.pool_size,
#             'init_norm': self.init_norm,
#             'normalize': self.normalize,
#         })
#         return config

#     def build(self, input_shape):
#         feature_size = input_shape[-1]
#         self.p = self.add_weight(name='norms', shape=(feature_size,),
#                                  initializer=tf.keras.initializers.constant(self.init_norm),
#                                  trainable=True)
#         super(GeM, self).build(input_shape)

#     def call(self, inputs):
#         x = inputs
#         x = tf.math.maximum(x, 1e-6)
#         x = tf.pow(x, self.p)

#         x = tf.nn.avg_pool(x, self.pool_size, self.pool_size, 'VALID')
#         x = tf.pow(x, 1.0 / self.p)

#         if self.normalize:
#             x = tf.nn.l2_normalize(x, 1)
#         return x

#     def compute_output_shape(self, input_shape):
#         return tuple([None, input_shape[-1]])
# class ArcMarginProduct(tf.keras.layers.Layer):
#     '''
#     Implements large margin arc distance.

#     Reference:
#         https://arxiv.org/pdf/1801.07698.pdf
#         https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
#             blob/master/src/modeling/metric_learning.py
#     '''
#     def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
#                  ls_eps=0.0, **kwargs):

#         super(ArcMarginProduct, self).__init__(**kwargs)

#         self.n_classes = n_classes
#         self.s = s
#         self.m = m
#         self.ls_eps = ls_eps
#         self.easy_margin = easy_margin
#         self.cos_m = tf.math.cos(m)
#         self.sin_m = tf.math.sin(m)
#         self.th = tf.math.cos(math.pi - m)
#         self.mm = tf.math.sin(math.pi - m) * m

#     def get_config(self):

#         config = super().get_config().copy()
#         config.update({
#             'n_classes': self.n_classes,
#             's': self.s,
#             'm': self.m,
#             'ls_eps': self.ls_eps,
#             'easy_margin': self.easy_margin,
#         })
#         return config

#     def build(self, input_shape):
#         super(ArcMarginProduct, self).build(input_shape[0])

#         self.W = self.add_weight(
#             name='W',
#             shape=(int(input_shape[0][-1]), self.n_classes),
#             initializer='glorot_uniform',
#             dtype='float32',
#             trainable=True,
#             regularizer=None)

#     def call(self, inputs):
#         X, y = inputs
#         y = tf.cast(y, dtype=tf.int32)
#         cosine = tf.matmul(
#             tf.math.l2_normalize(X, axis=1),
#             tf.math.l2_normalize(self.W, axis=0)
#         )
#         sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
#         phi = cosine * self.cos_m - sine * self.sin_m
#         if self.easy_margin:
#             phi = tf.where(cosine > 0, phi, cosine)
#         else:
#             phi = tf.where(cosine > self.th, phi, cosine - self.mm)
#         one_hot = tf.cast(
#             tf.one_hot(y, depth=self.n_classes),
#             dtype=cosine.dtype
#         )
#         if self.ls_eps > 0:
#             one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

#         output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
#         output *= self.s
#         return output
# def build_model(size=256, efficientnet_size=0, weights="imagenet", count=0):
#     inp = tf.keras.layers.Input(shape=(size, size, 3), name="inp1")
#     label = tf.keras.layers.Input(shape=(), name="inp2")
#     x = getattr(efn, f"EfficientNetB{efficientnet_size}")(
#         weights=weights, include_top=False, input_shape=(size, size, 3))(inp)
#     x = GeM(8)(x)
#     x = tf.keras.layers.Flatten()(x)
#     x = tf.keras.layers.Dense(512, name="dense_before_arcface", kernel_initializer="he_normal")(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = ArcMarginProduct(
#         n_classes=N_CLASSES,
#         s=30,
#         m=0.5,
#         name="head/arc_margin",
#         dtype="float32"
#     )([x, label])
#     output = tf.keras.layers.Softmax(dtype="float32")(x)
#     model = tf.keras.Model(inputs=[inp, label], outputs=[output])
#     opt = tf.optimizers.Adam(learning_rate=1e-4)
#     model.compile(
#         optimizer=opt,
#         loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
#         metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
#     )
#     return model
# def create_model_for_inference(weights_path: str):
#     with strategy.scope():
#         base_model = build_model(
#             size=256,
#             efficientnet_size=7,
#             weights=None,
#             count=0)
#         base_model.load_weights(weights_path)
#         model = tf.keras.Model(inputs=base_model.get_layer("inp1").input,
#                                outputs=base_model.get_layer("dense_before_arcface").output)
#         return model
# # Feature Extraction

# def to_hex(image_id) -> str:
#     return '{0:0{1}x}'.format(image_id, 16)


# def get_image_path(subset, image_id):
#     name = to_hex(image_id)
#     return os.path.join(DATASET_DIR, subset, name[0], name[1], name[2], '{}.jpg'.format(name))


# def load_image_tensor(image_path):
#     tensor = tf.convert_to_tensor(np.array(Image.open(image_path).convert("RGB")))
#     tensor = tf.image.resize(tensor, size=(256, 256))
#     tensor = tf.expand_dims(tensor, axis=0)
#     return tf.cast(tensor, tf.float32) / 255.0


# def create_batch(files):
#     images = []
#     for f in files:
#         images.append(load_image_tensor(f))
#     return tf.concat(images, axis=0)

# def extract_global_features_tf(df, image_root_dir, n_models=4):
#     image_paths = []
#     for i in range(len(df)):
#         tmp = df.loc[i,'id']
#         image_paths.append(os.path.join(image_root_dir, tmp[0], tmp[1],tmp[2],tmp+'.jpg'))
        
# #     for root, dirs, files in os.walk(image_root_dir):
# #         for file in files:
# #             if file.endswith('.jpg'):
# #                  image_paths.append(os.path.join(root, file))
                    
#     num_embeddings = len(image_paths)

#     ids = num_embeddings * [None]
#     ids = []
#     for path in image_paths:
#         ids.append(path.split('/')[-1][:-4])
    
#     embeddings = np.zeros((num_embeddings, 512))
#     image_paths = np.array(image_paths)
#     chunk_size = 512
    
#     n_chunks = len(image_paths) // chunk_size
#     if len(image_paths) % chunk_size != 0:
#         n_chunks += 1

#     for n in range(n_models):
#         print(f"Getting Embedding for fold{n} model.")
#         model = create_model_for_inference(f"../input/glret21-effb7/b7_fold{n}.h5")
#         for i in tqdm(range(n_chunks)):
#             files = image_paths[i * chunk_size:(i + 1) * chunk_size]
#             batch = create_batch(files)
#             embedding_tensor = model.predict(batch)
#             embeddings[i * chunk_size:(i + 1) * chunk_size] += embedding_tensor / n_models
#         del model
#         gc.collect()
#         tf.keras.backend.clear_session()

#     embeddings = normalize(embeddings, axis=1)

#     return ids, embeddings


# Config

In [5]:
import os, sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

class Config:
    batch_size = 64
    num_workers = 2
    num_to_rerank = 50
    seed = 717171
    embed_dim = 1024
    CACHE_DIR = '/kaggle/temp/'
    
hyperparameters = Config()
os.makedirs(hyperparameters.CACHE_DIR, exist_ok = True)
TOPK = 50

# Installation

In [6]:
!pip install ../input/pkg-wheels/einops-0.3.2-py3-none-any.whl
!pip install ../input/pkg-wheels/faiss_gpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl
!cp -r ../input/loftr-repo/ /kaggle/temp/
sys.path.append('/kaggle/temp/loftr-repo/')

Processing /kaggle/input/pkg-wheels/einops-0.3.2-py3-none-any.whl
Processing /kaggle/input/pkg-wheels/faiss_gpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl


# Imports

In [7]:
from scipy.spatial import cKDTree
from scipy import spatial
import pydegensac
import copy
import os
import numpy as np # linear algebra
import pandas as pd 
import random
from collections import defaultdict
from tqdm.auto import tqdm
import torch, cv2
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import warnings, math
from torch.nn.parameter import Parameter
import albumentations as A
import timm, gc
from sklearn.cluster import DBSCAN as dbscan
from src.loftr import LoFTR, default_cfg
import csv, shutil, glob, pickle, joblib
# from memory_profiler import profile
warnings.filterwarnings("ignore")

def seed_torch(seed=hyperparameters.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch()

# Dataset

In [8]:
class LandmarkDataset(Dataset):
    def __init__(self,image_ids, mode = 'train', prob_type = 'retrieval'):
        self.image_ids = image_ids
        
        if mode in ('train', 'index', 'test'):
#             self.file_path = f'../input/landmark-recognition-2021/{mode}'
            self.file_path = f'../input/landmark-{prob_type}-2021/{mode}'
        elif mode == 'nolandmark':
            self.file_path = f'../input/google-landmark-2021-validation/valid'
        else:
            raise
            
        self.transform = self._build_augmentation()
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        file_     = self.image_ids[idx]
        file_path = f'{self.file_path}/{file_[0]}/{file_[1]}/{file_[2]}/{file_}.jpg'
        raw_image = cv2.imread(file_path)
        img       = self.process_img(cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB))
        img       = img.transpose(2,0,1)
        return torch.from_numpy(img)
    
    def process_img(self, img):
        if self.transform is not None:
            img = self.transform(image = img)['image']
        img = img.astype(np.float32)
        img = self.normalize(img)
        return img
        
    def normalize(self, img):
        mean = np.array([123.675, 116.28 , 103.53 ], dtype=np.float32)
        std = np.array([58.395   , 57.120, 57.375   ], dtype=np.float32)
        img = img.astype(np.float32)
        img -= mean
        img *= np.reciprocal(std, dtype=np.float32)
        return img
    
    def _build_augmentation(self):
        return  A.Compose([
                A.SmallestMaxSize(512),
                A.CenterCrop(height=448,width=448,p=1.)
            ])
    
class GLRTRFDataset(Dataset):
    def __init__(self, feat, pair_tuples):
        self.feat = feat
        self.pair_tuples = pair_tuples
        return None
    
    def __getitem__(self, index:int):
        sample      = self.pair_tuples[index]
        query_emb   = self.feat[sample[0][0]:sample[0][0]+1]
        gallary_emb = self.feat[sample[0][1]:sample[0][1]+1]
        features    = np.expand_dims(np.array(sample[1]), 0)
        label       = np.expand_dims(np.array(sample[2]), 0)
        del sample
        return query_emb, gallary_emb, features, label
    
    def __len__(self,):
        return len(self.pair_tuples)
    
class GraphDataset(Dataset):

    def __init__(self, feats=None, labels=None, weights=None, pair_tuples=None, k=50, top_neighbors=None):
        self.feats = feats
        self.labels = labels
        self.weights = weights
        self.pair_tuples = pair_tuples
        self.k = k
        self.top_neighbors = top_neighbors

    def __getitem__(self, index):
        i, j = self.pair_tuples[index]
        feat = torch.FloatTensor(self.feats[i][j])

        padding_i = [[0] * feat.shape[0]] * (self.k - len(self.top_neighbors[i]))
        neighbor_feats_i = torch.FloatTensor([
            self.feats[i][neighbor]
            for neighbor in self.top_neighbors[i]
        ] + padding_i)
        padding_j = [[0] * feat.shape[0]] * (self.k - len(self.top_neighbors[j]))
        neighbor_feats_j = torch.FloatTensor([
            self.feats[j][neighbor]
            for neighbor in self.top_neighbors[j]
        ] + padding_j)
        neighbor_feats = torch.cat([feat.unsqueeze(0), neighbor_feats_i, neighbor_feats_j], dim=0)

        outputs = (feat, neighbor_feats)
        if self.labels is not None:
            outputs += (self.labels[i] == self.labels[j],)
        if self.weights is not None:
            outputs += (self.weights[i],)

        return outputs

    def __len__(self):
        return len(self.pair_tuples)

# CNN Model

## Metric Learning Layers

In [9]:
class ArcMarginProduct2(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(out_features, in_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.W)

    def forward(self, features):
        cosine = F.linear(F.normalize(features), F.normalize(self.W))
        return cosine
    
def l2_norm(input, axis = 1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output
    
class CurricularFace(nn.Module):
    r"""Implement of CurricularFace (https://arxiv.org/pdf/2004.00288.pdf):
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        device_id: the ID of GPU where the model will be trained by model parallel. 
                       if device_id=None, it will be trained on CPU without model parallel.
        m: margin
        s: scale of outputs
    """

    def __init__(self, in_features, out_features, m=0.4, s=45.0):
        super(CurricularFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.kernel = Parameter(torch.Tensor(in_features, out_features))
        self.register_buffer("t", torch.zeros(1))

    def forward(self, embbedings, label = None):
        embbedings = l2_norm(embbedings, axis=1)
        kernel_norm = l2_norm(self.kernel, axis=0)
        cos_theta = torch.mm(embbedings, kernel_norm)
        return cos_theta

    
class ArcMarginProduct_subcenter(nn.Module):
    def __init__(self, in_features, out_features, k=3):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features*k, in_features))
        self.reset_parameters()
        self.k = k
        self.out_features = out_features
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        
    def forward(self, features):
        cosine_all = F.linear(F.normalize(features), F.normalize(self.weight))
        cosine_all = cosine_all.view(-1, self.out_features, self.k)
        cosine, _ = torch.max(cosine_all, dim=2)
        return cosine

## Pooling Layers

In [10]:
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)[:,:,0,0]

# class GeM(nn.Module):
#     def __init__(self, p=3, eps=1e-6):
#         super(GeM, self).__init__()
#         self.p = Parameter(torch.ones(1) * p)
#         self.eps = eps
#     def forward(self, x):
#         return gem(x, p=self.p, eps=self.eps)
#     def __repr__(self):
#         return (
#             self.__class__.__name__
#             + f"(p={self.p.data.tolist()[0]:.4f}, eps={str(self.eps)})"
#         )

class GridGeM(nn.Module):
    def __init__(self, p = 3, eps = 1e-6):
        super(GridGeM, self).__init__()
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps
    
    def forward(self, x):
        B, C, H, W = x.shape
        
        f_a, f_b = torch.split(x,[W//2, W//2],dim=-1)
        f_a, f_c = torch.split(f_a,[H//2, H//2],dim=-2)
        f_b, f_d = torch.split(f_b,[H//2, H//2],dim=-2)

        feats = torch.cat([gem(f_a, p=self.p, eps=self.eps),gem(f_b, p=self.p, eps=self.eps),gem(f_c, p=self.p, eps=self.eps),gem(f_d, p=self.p, eps=self.eps),], axis=1)
        
        return feats
        
    def __repr__(self):
        return (
            self.__class__.__name__
            + f"(p={self.p.data.tolist()[0]:.4f}, eps={str(self.eps)})"
        )
    
class GridGeMCosdist(nn.Module):
    def __init__(self, p = 3, eps = 1e-6):
        super(GridGeMCosdist, self).__init__()
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps
        self.neck = nn.AdaptiveAvgPool2d( (1,1) )
    
    def forward(self, x):
        B, C, H, W = x.shape
        
        f_a, f_b = torch.split(x,[W//2, W//2],dim=-1)
        f_a, f_c = torch.split(f_a,[H//2, H//2],dim=-2)
        f_b, f_d = torch.split(f_b,[H//2, H//2],dim=-2)
        f_a = f_a.mean(dim=-1).softmax(dim=-1)
        f_b = f_b.mean(dim=-1).softmax(dim=-1)
        f_c = f_c.mean(dim=-1).softmax(dim=-1)
        f_d = f_d.mean(dim=-1).softmax(dim=-1)

        feats = torch.stack([(1.0 - F.cosine_similarity(f_a, f_b, dim=-1)), (1.0 - F.cosine_similarity(f_a, f_c, dim=-1)),
                           (1.0 - F.cosine_similarity(f_a, f_d, dim=-1)), (1.0 - F.cosine_similarity(f_b, f_c, dim=-1)),
                          (1.0 - F.cosine_similarity(f_b, f_d, dim=-1)), (1.0 - F.cosine_similarity(f_c, f_d, dim=-1))]).view(-1,1)
        
        return feats
        
    def __repr__(self):
        return (
            self.__class__.__name__
            + f"(p={self.p.data.tolist()[0]:.4f}, eps={str(self.eps)})"
        )

## Mix it

In [11]:
class LandmarkNet(nn.Module):
    def __init__(self, out_feature, backbone='tf_efficientnet_b6_ns',  pretrained=True, pool_type = 'gem',metric ='arcface', sub_center = False):
        super(LandmarkNet, self).__init__()
        self.backbne_name = backbone
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.out_feature = out_feature

        if "efficientnet" in backbone:
            self.in_features = self.backbone.classifier.in_features
            self.backbone.global_pool = nn.Identity()
            self.backbone.classifier = nn.Identity()
        elif "nfnet" in backbone:
            self.in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()
        elif 'resnet' in backbone:
            self.in_features = self.backbone.fc.in_features
            self.backbone.global_pool = nn.Identity()
            self.backbone.fc = nn.Identity() 
            
        self.out_feature = out_feature
        
        if pool_type == 'gem':
            self.pooling =  GeM()
        elif pool_type == 'gridgem':
            self.pooling =  GridGeM()
            self.in_features *= 4
        elif pool_type == 'gridgemcosdist':
            self.pooling =  GridGeMCosdist()
            self.in_features *= 6
        else:
            self.pooling = nn.AdaptiveAvgPool2d(1)
            
        self.neck = nn.Sequential(
                nn.Linear(self.in_features, 512, bias=True),
                nn.BatchNorm1d(512),
                torch.nn.PReLU()
            )
        if metric == 'arcface':
            if sub_center:
                self.final = ArcMarginProduct_subcenter(512, self.out_feature)
            else:
                self.final = ArcMarginProduct2(512, self.out_feature)
        elif metric == 'curricular':
            self.final = CurricularFace(512, self.out_feature)

    def forward(self, x, get_embeddings = True):
        if "efficientnet" in self.backbne_name or "nfnet" in self.backbne_name:
            batch_size = x.shape[0]
            features = self.backbone(x)
            features0 = self.pooling(features).view(batch_size, -1)
            features = self.neck(features0)
            features0 = F.normalize(features0)
            features = F.normalize(features)

        if not get_embeddings:
            return self.final(features)
        return features0 , features, self.final(features)

# Transformer Model

In [12]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.scale = nn.Parameter(torch.ones(1))

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(
            0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.scale * self.pe[:x.size(0), :]
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)

class GLRTRFModel(nn.Module):
    def __init__(self, embed_dim=128, dropout_rate=0.2):
        super(GLRTRFModel, self).__init__()
        self.embed_dim = embed_dim
        self.cat = nn.Sequential(
            nn.Linear(embed_dim*2, embed_dim),
            nn.LayerNorm(embed_dim),
        ) 
        self.cat1 = nn.Sequential(
            nn.Linear(embed_dim*2, embed_dim),
            nn.LayerNorm(embed_dim),
        ) 
        
        self.transformer = nn.Transformer(nhead=8, d_model = embed_dim, num_encoder_layers= N_LAYER, num_decoder_layers= N_LAYER, dropout = dropout_rate)

        self.dropout = nn.Dropout(dropout_rate)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        
        self.mlp1 = nn.Linear(N_FEATURE, 1024)
        self.mlp2 = nn.ReLU()
        self.mlp3 = nn.Linear(1024, 512)
        self.mlp4 = nn.ReLU()
        self.mlp5 = nn.Linear(512, embed_dim)
        self.pred = nn.Linear(embed_dim*2, 1)
    
    def forward(self, query_embedding, gallary_embedding, features):
        device = query_embedding.device 
        
        x1 = self.mlp1(features)
        x1 = self.mlp2(x1)
        x1 = self.mlp3(x1)
        x1 = self.mlp4(x1)
        x1 = self.mlp5(x1)

        x = torch.cat([query_embedding,x1], axis=-1)
        x = self.cat(x)
        e = torch.cat([gallary_embedding, x1], axis=-1)
        e = self.cat1(e)
        
        x = x.permute(1, 0, 2)
        e = e.permute(1, 0, 2)
        
        att_mask = future_mask(x.size(0)).to(device)
        att_output = self.transformer( x,e, src_mask=att_mask, tgt_mask=att_mask, memory_mask = att_mask)
        att_output = self.layer_normal(att_output+e)
        att_output = att_output.permute(1, 0, 2)
        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = torch.cat([x,x1], axis = -1)
        x = self.pred(x)
        return x.squeeze(-1)

# GCN Model

In [13]:
class GraphAttentionLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout=0.6, alpha=0.2, concat=True):
        super().__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h):
        Wh = h @ self.W  # h.shape: (B, N, in_features), Wh.shape: (B, N, out_features)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(3))

        attention = F.softmax(e, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.bmm(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        B, N, D = Wh.shape

        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=1)
        Wh_repeated_alternating = Wh.repeat(1, N, 1)

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=2)
        return all_combinations_matrix.view(-1, N, N, 2 * D)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class GATPairClassifier(nn.Module):
    def __init__(self, nfeat, nhid=8, nclass=1, dropout=0.6, alpha=0.2, nheads=8, pooling='first'):
        super().__init__()
        self.dropout = dropout
        self.pooling = pooling

        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GraphAttentionLayer(nhid * nheads, nhid, dropout=dropout, alpha=alpha, concat=False)

        self.classifier = nn.Sequential(
            nn.Linear(nfeat + nhid, nhid),
            nn.PReLU(),
            nn.BatchNorm1d(nhid),
            nn.Linear(nhid, nclass),
        )

    def forward_gat(self, x):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x) for att in self.attentions], dim=2)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x))
        if self.pooling == 'first':
            return x[:, 0]
        elif self.pooling == 'mean':
            return x.mean(dim=1)

    def forward(self, feats, neighbor_feats):
        gat_feats = self.forward_gat(neighbor_feats)
        cat_feats = torch.cat([feats, gat_feats], dim=1)
        return self.classifier(cat_feats).squeeze(1)

# Utils

In [14]:
def get_total_score(global_score, local_score, transformer_score, gcn_score, lgb_score, weights = [1]*5):
    classifier_score = (transformers_score * weights[2] + gcn_score * weights[3] + lgb_score * weights[4])
    score = (global_score * classifier_score * weights[0]) + ((local_score/MAX_INLIER_SCORE) * classifier_score * weights[0])
    return score

## Data Func

In [15]:
def load_labelmap(TRAIN_LABELMAP_PATH):
    with open(TRAIN_LABELMAP_PATH, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        labelmap = {row['id']: row['landmark_id'] for row in csv_reader}
    gc.collect()
    return labelmap

def memory_cleanup():
    """
    Cleans up GPU memory
    https://github.com/huggingface/transformers/issues/1742
    """
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            del obj

    gc.collect()
    torch.cuda.empty_cache()

## Submission Util

In [16]:
def save_submission_csv(DATASET_DIR, test_ids = None, predictions=None, mode = 'recognition',df_pair = None, submit_fname = 'submission.csv'):
    """Saves optional `predictions` as submission.csv.

      The csv has columns {id, landmarks}. The landmarks column is a string
      containing the label and score for the id, separated by a ws delimeter.

      If `predictions` is `None` (default), submission.csv is copied from
      sample_submission.csv in `IMAGE_DIR`.

      Args:
        predictions: Optional dict of image ids to dicts with keys {class, score}.
      """
    print('Submitting ..... ')
    if mode == 'retrieval':
        predicitions = defaultdict(list)
        for ix, i in enumerate(predictions):
            tmp = []
            for j in range(len(i)):
                tmp.append(i[j][0])
            predicitions[test_ids[ix]] = tmp
    elif mode == 'recognition':
        predicitions = defaultdict(lambda: defaultdict())
        for ix, i in enumerate(predictions):
            tmp = {}
#             for j in range(len(i)):
            tmp['class'] = i[0][1]
            tmp['score'] = i[0][2]
            predicitions[test_ids[ix]] = tmp
            
    gc.collect()
    if predictions is None:
    # Dummy submission!
        shutil.copyfile(
            os.path.join(DATASET_DIR, 'sample_submission.csv'), 'submission.csv')
        return
    
    if mode == 'recognition':
        with open(submit_fname, 'w') as submission_csv:
            csv_writer = csv.DictWriter(submission_csv, fieldnames=['id', 'landmarks'])
            csv_writer.writeheader()
            for ix,(image_id, prediction) in enumerate(predicitions.items()):
                label = prediction['class']
                score = prediction['score']
                csv_writer.writerow({'id': image_id, 'landmarks': f'{label} {score}'})
        gc.collect()
        return submit_fname
    else:
        with open(submit_fname, 'w') as submission_csv:
            csv_writer = csv.DictWriter(submission_csv, fieldnames=['id', 'images'])
            csv_writer.writeheader()
            for image_id, prediction in predicitions.items():
                image_ids = prediction
                csv_writer.writerow({'id': image_id, 'images': ' '.join(image_ids)})
        gc.collect()
        return submit_fname

In [17]:
# def save_submission_csv(DATASET_DIR,df_pair, test_ids = None, predictions=None, mode = 'recognition', submit_fname = 'submission.csv'):
#     """Saves optional `predictions` as submission.csv.

#       The csv has columns {id, landmarks}. The landmarks column is a string
#       containing the label and score for the id, separated by a ws delimeter.

#       If `predictions` is `None` (default), submission.csv is copied from
#       sample_submission.csv in `IMAGE_DIR`.

#       Args:
#         predictions: Optional dict of image ids to dicts with keys {class, score}.
#       """
#     print('Submitting ..... ')
#     if mode == 'retrieval':
#         predicitions = defaultdict(lambda: defaultdict())
#         for ix, i in enumerate(predictions):
#             tmp = {}
#             for j in range(len(i)):
#                 s = df_pair[(df_pair.id == test_ids[ix]) & (df_pair.id_target == i[j][0])]['pred'].values
#                 tmp['target_id'] = i[j][0]
#                 tmp['score'] = i[j][2]*s
#             predicitions[test_ids[ix]] = tmp
#         for (image_id, pr) in predicitions.items():
#             predicitions[image_id] = sorted(pr, key = lambda x: x['score'])
        
#     elif mode == 'recognition':
#         predicitions = defaultdict(lambda: defaultdict())
#         for ix, i in enumerate(predictions):
#             tmp = {}
#             for j in range(len(i)):
#                 s = df_pair[(df_pair.id == test_ids[ix]) & (df_pair.id_target == i[j][0])]['pred'].values
#                 tmp['class'] = i[j][1]
#                 tmp['score'] = i[j][2]*s
#             predicitions[test_ids[ix]] = tmp

#         for image_id, pr in predicitions.items():
#             predicitions[image_id] = sorted(pr, key = lambda x: x['score'])
        
            
#     gc.collect()
#     if predictions is None:
#     # Dummy submission!
#         shutil.copyfile(
#             os.path.join(DATASET_DIR, 'sample_submission.csv'), 'submission.csv')
#         return
    
#     if mode == 'recognition':
#         with open(submit_fname, 'w') as submission_csv:
#             csv_writer = csv.DictWriter(submission_csv, fieldnames=['id', 'landmarks'])
#             csv_writer.writeheader()
#             for ix,(image_id, prediction) in enumerate(predicitions.items()):
#                 label = prediction['class']
#                 score = prediction['score']
#                 csv_writer.writerow({'id': image_id, 'landmarks': f'{label} {score}'})
#         gc.collect()
#         return submit_fname
#     else:
#         with open(submit_fname, 'w') as submission_csv:
#             csv_writer = csv.DictWriter(submission_csv, fieldnames=['id', 'images'])
#             csv_writer.writeheader()
#             for image_id, prediction in predicitions.items():
#                 image_ids = prediction
#                 csv_writer.writerow({'id': image_id, 'images': ' '.join(image_ids)})
#         gc.collect()
#         return submit_fname

## Global Feature Extraction

In [18]:
def extract_global_features(df, mode, prob_type, return_probs = False):
    dataset     = LandmarkDataset(df['id'].values, mode = mode, prob_type = prob_type)
    dataloader  = DataLoader(dataset, batch_size = hyperparameters.batch_size, shuffle=False, num_workers=hyperparameters.num_workers)
    feats = []
    with torch.no_grad():
        for img in tqdm(dataloader):
            img  = img.cuda()
            feat = []
            for model in models:
                _,feat_m,_  = model(img)
                feat.append(feat_m)
            feat = torch.cat(feat,dim=1) 
#             feat = torch.sum(torch.stack([feat]), dim = 0)
            
#             if return_probs:
#                 probs_m = (lo_1 + lo_2)/2
#                 save_pickle(probs_m,f'{mode}_probs.pkl')
                
            feats.append(feat.detach().cpu())
        feats = torch.cat(feats)
        feats = feats.cuda()
        feat  = F.normalize(feat) 
#         save_pickle(feats,f'{mode}_embedding')
    del feat, img, dataset, dataloader
    gc.collect()
    if return_probs:
        return df['id'].values, feats, prob_m
    return df['id'].values,feats.detach().cpu().numpy()

## LoFTR Model

In [19]:
def generate_loftr(matcher, image_0, train_images):
    img0 = []
    img1 = []
    img0_scales = []
    img1_scales = []
    for image_1 in train_images:
        img0_raw = cv2.imread(image_0, cv2.IMREAD_GRAYSCALE)
        img1_raw = cv2.imread(image_1, cv2.IMREAD_GRAYSCALE)
        img0_scales.append((img0_raw.shape[1]/640, img0_raw.shape[0]/480))
        img1_scales.append((img1_raw.shape[1]/640, img1_raw.shape[0]/480))
        img0.append(cv2.resize(img0_raw, (640, 480)))
        img1.append(cv2.resize(img1_raw, (640, 480)))
        
    del img0_raw, img1_raw
        
    img0 = torch.from_numpy(np.array(img0)[:,None,...]).cuda() / 255.
    img1 = torch.from_numpy(np.array(img1)[:,None,...]).cuda() / 255.
    batch = {'image0': img0, 'image1': img1}
    del img0 , img1
    with torch.no_grad():
        matcher(batch)
    del batch['image0'], batch['image1']
    batch['img0_scales'] = img0_scales
    batch['img1_scales'] = img1_scales
    gc.collect()
    del matcher
    return batch
    
def rescore_and_rerank(test_image_dir, train_image_dir,
                                      test_image_id, train_ids_labels_and_scores, batch_size = 16, ignore_global_score=False, do_sort=True,
                                      loftr_model=None, return_num_inliers=False):
    
    test_image_path = f'{test_image_dir}/{test_image_id[0]}/{test_image_id[1]}/{test_image_id[2]}/{test_image_id}.jpg'
    test_image_dict = {}

    ransac_inliers = []
    
    train_images_path = []
    
    for i in range(len(train_ids_labels_and_scores)):
        train_image_id, label, global_score = train_ids_labels_and_scores[i]
        train_images_path.append(f'{train_image_dir}/{train_image_id[0]}/{train_image_id[1]}/{train_image_id[2]}/{train_image_id}.jpg')
        
    for batch_i in range(0,len(train_images_path), batch_size):
        pred = generate_loftr(loftr_model,test_image_path,train_images_path[batch_i:batch_i+batch_size] )
        test_scales = pred['img0_scales'][0]
        test_keypoints = copy.deepcopy(pred['mkpts0_f']).cpu().numpy()*test_scales[0]
        m_bids = pred['m_bids'].cpu().numpy()
        del test_scales
        up = batch_size if batch_i + batch_size < len(train_images_path) else len(train_images_path) - batch_i
        for i in range(up):
            train_image_id, label, global_score = train_ids_labels_and_scores[batch_i + i]
            train_scales = pred['img1_scales'][i]
            train_keypoints = copy.deepcopy(pred['mkpts1_f'][m_bids == i]).cpu().numpy()*train_scales
            num_inliers = compute_num_inliers(test_keypoints[m_bids == i], None, train_keypoints, None, do_kdtree=False)

            total_score = global_score + num_inliers/MAX_INLIER_SCORE
            train_ids_labels_and_scores[batch_i + i] = (train_image_id, label, total_score)
            ransac_inliers.append((train_image_id, num_inliers))
        del pred, test_keypoints, train_keypoints, train_scales
        
    if do_sort:
        train_ids_labels_and_scores.sort(key=lambda x: x[2], reverse=True)
    gc.collect()   
    if return_num_inliers:
        return ransac_inliers
    else:
        return train_ids_labels_and_scores

## Others

In [20]:
def do_gcn(df_train,df_test, gcn_rerank = 25, topk = 15):
    
    indexes_img = load_pickle('ind_train_test_sim.npy')
    similarities_img = load_pickle('val_train_test_sim.npy')
    row = indexes_img.ravel()
    col = np.arange(len(indexes_img)).repeat(topk)
    data = similarities_img.ravel()
    
    simmat_img = {(i, j): d for i, j, d in zip(col, row, data)}
    
    top_neighbors = defaultdict(list)
    feats = defaultdict(lambda: defaultdict())
    pair_tuples = []
    for i in tqdm(range(len(df_test))):
        right_indexes = set(indexes_img[i, :gcn_rerank].tolist())

        right_indexes = list(right_indexes)
        scores = {}
        for j in right_indexes:
            pair_tuples.append((i, j))

            sim_img = simmat_img.get((i, j), 0)
            if sim_img == 0:
                print('.',end = '')
                continue

            feats[i][j] = [
                sim_img,
            ]
            scores[j] = sim_img

        top_neighbors[i] = sorted(right_indexes, key=lambda x: scores[x], reverse=True)[:gcn_rerank]

    dataset = GraphDataset(
        feats=feats,
        pair_tuples=pair_tuples,
        k=15,
        top_neighbors=top_neighbors,
    )
    loader = DataLoader(dataset, batch_size=2 ** 6, shuffle=False, drop_last=False, num_workers=2, pin_memory=True)

    gat = GATPairClassifier(nfeat=1, nhid=16, dropout=0.0, nheads=16, pooling='first')
    gat.to('cuda').eval()
    gat.load_state_dict(torch.load('../input/model/glrtrf_model_last.pt'))

    gc.collect()
    
    preds = []
    for feats, neighbor_feats in tqdm(loader, desc='predict', leave=False):
        feats = feats.to('cuda', non_blocking=True)
        neighbor_feats = neighbor_feats.to('cuda', non_blocking=True)
        with torch.no_grad():
            pred = gat(feats, neighbor_feats).sigmoid().detach().cpu().numpy().tolist()
            preds.extend(pred)
    
    
    df_pair = pd.DataFrame()
    df_pair['i'] = col
    df_pair['j'] = row

    df_pair['id'] = df_test['id'].values[df_pair['i'].values]
    df_pair['id_target'] = df_train['id'].values[df_pair['j'].values]
    df_pair['pred'] = preds
    del gat, loader, dataset, top_neighbors, scores, feats, right_indexes, pair_tuples, col, row, data, indexes_img, similarities_img
    return df_pair

In [21]:
def combine(d1, d2, i1,i2):
    for i in range(d1.shape[0]):
        for j in range(d1.shape[1]):
            for k in range(d2.shape[1]):
                if i2[i,k] == i1[i, j]:
                    if d1[i, j] < 0:
                        d1[i, j] *= (1 - d2[i,k])
                    else:
                        d1[i, j] *= d2[i,k]
    
                        
    return d1, i1

In [22]:
def query_expansion(feats, sims, topk_idx, alpha=0.5, k=2):
    weights = np.expand_dims(sims[:, :k] ** alpha, axis=-1).astype(np.float32)
    feats   = (feats[topk_idx[:, :k]] * weights).sum(axis=1)
    feats  /= np.linalg.norm(feats, 2, axis=1, keepdims=True)
    return feats

def do_retrieval(labelmap, train_ids, train_embeddings,
                 test_embeddings, num_to_rerank,df_pair = None, val_x = None, val_y = None, do_dba=False,
                 gallery_set='index', method = 'cossim',return_vals = False,lgb = False,save = False, name ='simple'):
    train_ids_labels_and_scores = [None] * test_embeddings.shape[0]
    
    if method == 'faiss':
        res = faiss.StandardGpuResources()
        faiss_index = faiss.IndexFlatIP(train_embeddings.shape[1])
        faiss_index = faiss.index_cpu_to_gpu(res, 0, faiss_index)
        faiss_index.add(train_embeddings)
        D, I = faiss_index.search(test_embeddings, num_to_rerank)  # actual search
        
    elif method == 'cossim':
        if val_x is None:
            D, I = get_topk_cossim(test_embeddings, train_embeddings, batchsize = 64, k=num_to_rerank, device='cuda:0',verbose=True,save = save, name = name)
        else:
            D, I = get_topk_cossim_sub(test_embeddings, train_embeddings,val_x, batchsize = 64, k=num_to_rerank, device='cuda:0',verbose=True,save = save, name = name)
        if val_y is not None:
            D -= 1* val_y
    
    for test_index in range(test_embeddings.shape[0]):
        train_ids_labels_and_scores[test_index] = [
          (train_ids[train_index], labelmap[train_ids[train_index]], distance)
          for train_index, distance in zip(I[test_index], D[test_index])
        ]
    del I
    gc.collect()
    if return_vals:
        return train_ids_labels_and_scores, D
    del D
    return train_ids_labels_and_scores

def get_nolandmark_by_dbscan(test_ids, test_embeddings, nolandmark_ids, nolandmark_embeddings):
  # dbscan
    features = np.vstack([test_embeddings, nolandmark_embeddings])
    clusters = dbscan(eps=0.85, n_jobs=-1, min_samples=1).fit_predict(features)
    clusters_np = np.c_[np.r_[test_ids, nolandmark_ids], clusters]
    clusters_df = pd.DataFrame(data=clusters_np, columns=[ID, 'clusters'])
    clusters_df['is_nolandmark'] = [0]*len(test_ids) + [1]*len(nolandmark_ids)
    clusters_gb = clusters_df.groupby('clusters')['is_nolandmark'].agg(['count', 'sum']).reset_index()
    clusters_gb.columns = ['clusters', 'clusters_num', 'nolandmark_num']
    clusters_gb['nolandmark_rate'] = clusters_gb['nolandmark_num'] / clusters_gb['clusters_num']

    test_clusters = clusters_df[0: len(test_ids)]
    test_clusters = test_clusters.merge(clusters_gb, on='clusters', how='left')
    gc.collect()
    return test_clusters

## CosSim

In [23]:
def cos_similarity_matrix(a, b, eps=1e-8):
    sim_mt = torch.mm(a, b.transpose(0, 1))
    return sim_mt

def get_topk_cossim(test_emb, tr_emb, batchsize = 64, k=10, device='cuda:0',verbose=True, save = False, name = 'simple'):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []
    for test_batch in tqdm(test_emb.split(batchsize),disable=1-verbose):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    if save:
        save_pickle(vals,'val_'+name)
        save_pickle(inds,'ind_'+name)
    gc.collect()
    return vals, inds

def get_topk_cossim_sub(test_emb, tr_emb, vals_x, batchsize = 64, k=10, device='cuda:0',verbose=True,save = False, name = 'simple'):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    vals_x = torch.tensor(vals_x, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []
    for test_batch in tqdm(test_emb.split(batchsize),disable=1-verbose):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        sim_mat = torch.clamp(sim_mat,0,1) - vals_x.repeat(sim_mat.shape[0], 1)
        
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    if save:
        save_pickle(vals,'val_'+name)
        save_pickle(inds,'ind_'+name)
    gc.collect()
    return vals, inds

## Util

In [24]:
def save_pickle(data, filename = 'tmp', save_joblib = False):
    print('\tSaving {} to {}'.format(filename.split('.')[0], filename))
    try:
        np.save(os.path.join(hyperparameters.CACHE_DIR,filename), data)
    except:
        np.save(os.path.join(hyperparameters.CACHE_DIR,filename), data.detach().cpu().numpy())
    
    del data
    
def load_pickle(filename):
    print('\tLoading {} from {}'.format(filename.split('.')[0], filename))
    return np.load(os.path.join(hyperparameters.CACHE_DIR,filename))

def index_df():
    image_ids = []
    for i in glob.glob('../input/landmark-retrieval-2021/index/*/*/*/*.jpg'):
        image_ids.append(i.split('/')[-1].split('.')[0])
    df = pd.DataFrame({'id':image_ids, 'landmark_id': [-2]*len(image_ids)})
    del image_ids
    return df

def targets_prob():
    p = np.load('../input/glr-files/classes.npy')
    idx2landmark_id = {ix:x for ix, x in enumerate(p)}
    landmark_id2idx = {x:ix for ix, x in enumerate(p)}
    del p

    pred_mask = pd.Series(df_train.landmark_id.unique()).map(landmark_id2idx).values
    gc.collect()
    return pred_mask, idx2landmark_id, landmark_id2idx 

## Compute inlier

In [25]:
MAX_INLIER_SCORE = 70
MAX_REPROJECTION_ERROR = 4.0
MAX_RANSAC_ITERATIONS = 1000
HOMOGRAPHY_CONFIDENCE = 0.99

def compute_putative_matching_keypoints(test_keypoints,
                                        test_descriptors,
                                        train_keypoints,
                                        train_descriptors,
                                        max_distance=0.9):
    """Finds matches from `test_descriptors` to KD-tree of `train_descriptors`."""

    train_descriptor_tree = spatial.cKDTree(train_descriptors)
    _, matches = train_descriptor_tree.query(
      test_descriptors, distance_upper_bound=max_distance)

    test_kp_count = test_keypoints.shape[0]
    train_kp_count = train_keypoints.shape[0]

    test_matching_keypoints = np.array([
      test_keypoints[i,]
      for i in range(test_kp_count)
      if matches[i] != train_kp_count
    ])
    train_matching_keypoints = np.array([
      train_keypoints[matches[i],]
      for i in range(test_kp_count)
      if matches[i] != train_kp_count
    ])
    gc.collect()
    return test_matching_keypoints, train_matching_keypoints

def compute_num_inliers(test_keypoints, test_descriptors, train_keypoints,
                        train_descriptors, do_kdtree=True):
    """Returns the number of RANSAC inliers."""

    if do_kdtree:
        test_match_kp, train_match_kp = compute_putative_matching_keypoints(
            test_keypoints, test_descriptors, train_keypoints, train_descriptors)
    else:
        test_match_kp, train_match_kp = test_keypoints, train_keypoints
    if test_match_kp.shape[0] <= 4:  # Min keypoints supported by `pydegensac.findHomography()`
        return 0

    try:
        _, mask = pydegensac.findHomography(test_match_kp, train_match_kp,
                                        MAX_REPROJECTION_ERROR,
                                        HOMOGRAPHY_CONFIDENCE,
                                        MAX_RANSAC_ITERATIONS)
    except np.linalg.LinAlgError:  # When det(H)=0, can't invert matrix.
        return 0
    gc.collect()
    return int(copy.deepcopy(mask).astype(np.float32).sum())

# Load_model

In [26]:
def load_model(model, model_file):
    state_dict = torch.load(model_file)
    if "model_state_dict" in state_dict.keys():
        state_dict = state_dict["model_state_dict"]
    state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
#     del state_dict['final.weight']
    model.load_state_dict(state_dict, strict=True)
    print(f"loaded {model_file}")
    del state_dict
    model.eval()  
    gc.collect()
    return model

# model1 = LandmarkNet(81313, backbone = 'tf_efficientnet_b3_ns', pool_type = 'gridgem', metric = 'curricular', pretrained = False)
# model1 = load_model(model1,'../input/google-mark-no1/epoch_6_score_0.01662521950018494.pth').cuda()
model1 = LandmarkNet(203094, backbone = 'tf_efficientnetv2_m_in21ft1k', pool_type = 'gridgem', metric = 'arcface', pretrained = False)
model1 = load_model(model1,'../input/model/epoch_5_score_0.0.pth').cuda()

# model2 = LandmarkNet(81313, backbone = 'tf_efficientnetv2_m_in21ft1k', pool_type = 'gridgem', metric = 'arcface', pretrained = False)
# model2 = load_model(model2,'../input/model/epoch_9_score_0.1616932213254062 (1).pth').cuda()

# model3 = LandmarkNet(81313, backbone = 'tf_efficientnet_b3_ns', pool_type = 'gridgem', metric = 'curricular', pretrained = False)
# model3 = load_model(model3,'../input/model/epoch_10_score_0.16941185050639018.pth').cuda()

model4 = LandmarkNet(203094, backbone = 'tf_efficientnet_b3_ns', pool_type = 'gridgem', metric = 'curricular', pretrained = False)
model4 = load_model(model4,'../input/model/epoch_3_score_0.0b3ns.pth').cuda()

# model1 = LandmarkNet(81313, backbone = 'tf_efficientnet_b3', pool_type = 'gem',sub_center = True, pretrained = False)
# model1 = load_model(model1,'../input/morizin-exp1/epoch_10_score_0.001845540972934233.pth').cuda()
# model2 = LandmarkNet(81313, backbone = 'tf_efficientnet_b3', pool_type = 'gem',sub_center = True, pretrained = False)
# model2 = load_model(model2,'../input/morizin-exp1/epoch_9_score_0.0015537621993018456.pth').cuda()
# models = [model1, model2, model3, model4]
models = [model1, model4,]
del model1, model4

loaded ../input/model/epoch_5_score_0.0.pth
loaded ../input/model/epoch_3_score_0.0b3ns.pth


# Inference

In [27]:
def get_prediction(mode = 'retrieval'):
    if mode == 'retrieval':
        train_image_dir = f'../input/landmark-retrieval-2021/index'
        test_image_dir = f'../input/landmark-retrieval-2021/test'
    elif mode == 'recognition':
        train_image_dir = f'../input/landmark-recognition-2021/train'
        test_image_dir = f'../input/landmark-recognition-2021/test'
    
    df_train = pd.read_csv(f'../input/landmark-{mode}-2021/train.csv')
    df_test  = pd.read_csv(f'../input/landmark-{mode}-2021/sample_submission.csv')
    df_nl    = pd.read_csv('../input/google-landmark-2021-validation/valid.csv').sample(50000).reset_index(drop = True)
    
    gc.collect()
    
    if mode == 'retrieval':
        if len(df_test) == 1129:
            shutil.copy('../input/landmark-retrieval-2021/sample_submission.csv','./submission.csv')
            return './submission.csv'
        
        df_index = index_df()
        train_ids, train_embeddings = extract_global_features(df_index, mode = 'index', prob_type = mode)
#         _, train_embeddings2 = extract_global_features_tf(df_index, train_image_dir)
        del df_index
    else:
        if len(df_test) == 10345:
            shutil.copy('../input/landmark-recognition-2021/sample_submission.csv','./submission.csv')
            return './submission.csv'
        train_ids, train_embeddings = extract_global_features(df_train, mode = 'train', prob_type = mode)
#         _, train_embeddings2 = extract_global_features_tf(df_train, train_image_dir)
    
    nolandmark_ids, nolandmark_embeddings = extract_global_features(df_nl, mode = 'nolandmark', prob_type = mode)
#     _, nolandmark_embeddings2 = extract_global_features_tf(df_nl, '../input/google-landmark-2021-validation/valid')
    
    test_ids, test_embeddings = extract_global_features(df_test, mode = 'test', prob_type = mode)
#     _, test_embeddings2 = extract_global_features_tf(df_test,test_image_dir)
    
#     train_embeddings = np.concatenate([train_embeddings, train_embeddings2], axis = 1)
#     nolandmark_embeddings = np.concatenate([nolandmark_embeddings, nolandmark_embeddings2], axis = 1)
#     test_embeddings = np.concatenate([test_embeddings, test_embeddings2], axis = 1)
    
    del  df_nl
#     , train_embeddings2, test_embeddings2, nolandmark_embeddings2
    memory_cleanup()
    del globals()['models']
    
    if mode == 'retrieval':
        labelmap = dict([(i, -2) for i in train_ids])
    else:
        labelmap = load_labelmap('../input/landmark-recognition-2021/train.csv')
    nolandmark_labelmap = dict([(i, -1) for i in nolandmark_ids])
    
    memory_cleanup()
    
    _, val_x = do_retrieval(nolandmark_labelmap, nolandmark_ids,
                                                    nolandmark_embeddings, train_embeddings,
                                                    5, gallery_set='nolandmark',return_vals = True, name = 'train_nl_sim')
    
    _, val_y = do_retrieval(nolandmark_labelmap, nolandmark_ids,
                                                    nolandmark_embeddings, test_embeddings,
                                                    11, gallery_set='nolandmark',return_vals = True, name = 'test_nl_sim')
    del nolandmark_embeddings, nolandmark_ids, nolandmark_labelmap
    
    train_ids_labels_and_scores = do_retrieval(labelmap, train_ids,
                                               train_embeddings, test_embeddings,
                                               25, gallery_set='train', save = True, name= 'train_test_sim')
    
#     df_pair = do_gcn(df_train,df_test, 25, 25)
    del df_train,df_test
    
    train_ids_labels_and_scores = do_retrieval(labelmap, train_ids,
                                               train_embeddings, test_embeddings,
                                               3,val_x = val_x[:,:].mean(axis=1).detach().cpu().numpy(),
                                               val_y = None, gallery_set='train', name= 'train_test_sim')
    
    
    
    memory_cleanup()
    del train_embeddings, test_embeddings, val_y, val_x, labelmap
    memory_cleanup()
    submit_fname = save_submission_csv(f'/kaggle/input/landmark-{mode}-2021', test_ids = test_ids, predictions=train_ids_labels_and_scores, mode = mode,)
#     save_pickle(train_ids_labels_and_scores, 'predicition_v1.pkl')
    del train_ids_labels_and_scores
    memory_cleanup()
    return submit_fname

In [28]:
sub_v1    = pd.read_csv(get_prediction(mode = 'recognition'))

In [29]:
sub_v1

,id,landmarks
0,00084cdf8f600d00,137790 0.1
1,000b15b043eb8cf0,137790 0.1
2,0011a52f9b948fd2,137790 0.1
3,00141b8a5a729084,137790 0.1
4,0018aa4b92532b77,137790 0.1
...,...,...
10340,ffc41ddbcf63289b,137790 0.1
10341,ffd4c19b7732cbe9,137790 0.1
10342,ffee2cac79a173d6,137790 0.1
10343,ffef459d5dc6b981,137790 0.1
